# Quantum Sticks

### The Classical Version
The classical version of the "Sticks" game can be described as follows:
* There are a total of n sticks. 
* Each player can either pick 1, 2 or 3 sticks per turn. 
* The player with the current turn and no more sticks to pick loses. 

### Let's Quantumize It

We propose a quantum version of the same game- the description is as follows:
* There are a total of n sticks
* Each player can choose to either pick 1,2 or 3 sticks per turn. However, the architecture of the game may/may not allow those many sticks to be picked. The total number of sticks picked may increase/decrease. Goofy.

In order to achieve this 'goofy action', we have the below:
* We create n qubits- half of these are initialized to |0>, the others to |1>
* The user can choose to put some qubits in superposition
* We entangle some pairs of qubits in the back-end
* When a player picks m sticks, we measure those many sticks (and if there are any other sticks which are entangled to the selected sticks). Depending on the measurement outcome, we calculate how many sticks were actually picked.


In [513]:
from qiskit import *
import numpy as np
import pyfiglet
import time
import cowsay
import termcolor
import random
from IPython.display import clear_output

In [514]:
#Game title
termcolor.cprint(pyfiglet.figlet_format("Quantum Sticks" ),'green', attrs=['bold'])
#Authors
termcolor.cprint(pyfiglet.figlet_format(" Jaime  McCarthy ", font = "digital" ),'blue', 'on_grey')
termcolor.cprint(pyfiglet.figlet_format(" Kaushambi  Gujral ", font = "digital"),'magenta', 'on_grey')
termcolor.cprint(pyfiglet.figlet_format(" Lee  Hoang ", font = "digital"),'yellow', 'on_grey')

  ___                    _                     ____  _   _      _        
 / _ \ _   _  __ _ _ __ | |_ _   _ _ __ ___   / ___|| |_(_) ___| | _____ 
| | | | | | |/ _` | '_ \| __| | | | '_ ` _ \  \___ \| __| |/ __| |/ / __|
| |_| | |_| | (_| | | | | |_| |_| | | | | | |  ___) | |_| | (__|   <\__ \
 \__\_\\__,_|\__,_|_| |_|\__|\__,_|_| |_| |_| |____/ \__|_|\___|_|\_\___/
                                                                         

 +-+-+-+-+-+  +-+-+-+-+-+-+-+-+ 
 |J|a|i|m|e|  |M|c|C|a|r|t|h|y| 
 +-+-+-+-+-+  +-+-+-+-+-+-+-+-+ 

 +-+-+-+-+-+-+-+-+-+  +-+-+-+-+-+-+ 
 |K|a|u|s|h|a|m|b|i|  |G|u|j|r|a|l| 
 +-+-+-+-+-+-+-+-+-+  +-+-+-+-+-+-+ 

 +-+-+-+  +-+-+-+-+-+ 
 |L|e|e|  |H|o|a|n|g| 
 +-+-+-+  +-+-+-+-+-+ 



**Computer Strategies**

In [525]:
class Strategy:
    def __init__(self, strategy):
        self.strategy = strategy
        self.constant = random.randint(1,3)
        self.balance = 1
    
    def constant_strategy(self):
        return self.constant #always pick the same number of sticks
    
    def balanced_strategy(self):
        x = self.balance
        self.balance +=1
        return x % 3
    
    def calculated_strategy(self, numSticks):
        if numSticks%4 == 3:
            return 2
        elif numSticks%4 == 2:
            return 1
        elif numSticks%4 == 1:
            return 1
        elif numSticks%4 == 0:
            return 3
        
    def pick_sticks(self, sticks):
        if self.strategy == 1:
            return self.constant_strategy()
        elif self.strategy == 2:
            return self.balanced_strategy()
        else:
            return self.calculated_strategy(sticks)

In [526]:
print("Testing computer's strategies")


print("Strategy 1")
s = Strategy(1)
sticks_remaining = 12
x = s.pick_sticks(sticks_remaining)
print(x)
sticks_remaining -= x
print(s.pick_sticks(sticks_remaining))


print("Strategy 2")
s = Strategy(2)
sticks_remaining = 12
x = s.pick_sticks(sticks_remaining)
print(x)
sticks_remaining -= x
print(s.pick_sticks(sticks_remaining))


print("Strategy 3")
s = Strategy(3)
sticks_remaining = 12
x = s.pick_sticks(sticks_remaining)
print(x)
sticks_remaining -= x
print(s.pick_sticks(sticks_remaining))



Testing computer's strategies
Strategy 1
3
3
Strategy 2
1
2
Strategy 3
3
1


In [528]:
# To-do: Understand what strategy works with the architecture we've created.
# Can you win the game with some probability p?

class QuantumSticks:
    
    def __init__(self):
        self.turn = True
        self.welcome()
        self.min_sticks = 5 
        self.max_sticks = 25 # n>=5 && n<=25
        self.level = 0
        self.n = 25
        self.qr = None
        self.cr = None
        self.qc = None
        self.create_circuit()
        self.picked_sticks = []
        self.unpicked_sticks = (np.arange(0, 24, 1, dtype=int)).tolist()
        
    #---------------------
    #Utility Methods start
    #---------------------
    def clear(self, sleep_time):
        time.sleep(sleep_time)
        clear_output(wait=True)
        
    def slow_display(self, msg):
        for x in range (0,4):  
            b = msg + "." * x
            print (b, end="\r")
            time.sleep(0.8)
        print ("\n")
        
    #---------------------

    
    
    #---------------------
    #Core Methods start
    #---------------------
    def create_circuit(self):
        m = self.max_sticks
        self.qr = QuantumRegister(m)
        self.cr = ClassicalRegister(1)
        self.qc = QuantumCircuit(self.qr, self.cr)
        self.qc.reset(range(m))
        for i in range(0, m-1, 2):
            self.qc.x(i)
        
    def measure_specific(self, l):
        measurements = []
        for register in l:
            self.qc.measure(self.qr[register], self.cr)
            backend = Aer.get_backend('aer_simulator')
            job = backend.run(self.qc, shots=1024, memory=True)
            result = job.result()
            counts = result.get_counts(self.qc)
            zeros = counts.get('0', 0)
            ones = counts.get('1', 0)
            if int(zeros) > int(ones):
                measurements.append(0)
            else:
                measurements.append(1)
        return measurements  
    
    '''
        Apply cnot to alternate qubits
    '''    
    def cnot_alternate(self):
        for i in range(0, self.max_sticks-1, 2):
            self.qc.cx(i,i+1)    
        
    def draw_circuit(self):
        print(self.qc)
        
    ''' Puts some m sticks into superposition where m<=20. 
        Thus, every time, the game begins with a different n where n>=5 
    '''
    def initialize_sticks(self, superpositions): 
        self.qc.barrier()
        
        for s in superpositions:
            sup = int(s) - 1
            if sup>24 or sup<0:
                continue
            self.qc.h(sup)
        
        self.qc.barrier()
        
        
    def begin_game(self):
        print("You can put some sticks in superposition.")
        print("Please enter comma separated values: such as 1,12,5,21,..")
        s = input().split(',')[:25]
        sups = [*set(s)]
        self.initialize_sticks(sups)
        self.cnot_alternate()
        self.play_game()
        
    def pick_sticks(self, m):
        total_m = 0
        if len(self.unpicked_sticks) == 0:
            return -999
        for i in range(0, m):
            el = self.unpicked_sticks[0]
            # if el is even, measure el and el+1: this decides how many sticks you're picking
            if el%2==0:
                measurements = self.measure_specific([el, el+1])
                if measurements[0] == 1:
                    self.picked_sticks.append(el)
                    self.unpicked_sticks.remove(el)
                    total_m +=1
                if measurements[1] == 1:
                    self.picked_sticks.append(el+1)
                    self.unpicked_sticks.remove(el+1)
                    total_m +=1
            else:
                measurement = self.measure_specific([el])
                if measurement[0] == 1:
                    total_m +=1
        return total_m #the number of sticks that were actually picked

    def play_game(self):
        strategy = Strategy(1) #computer's strategy
        while self.n > 1:
            
            print('-------------------------------------------\nSticks left: %2d' %self.n)
            pick = 0
            if self.turn:
                # pick sticks
                print("Enter the number of sticks you want to pick- 1,2,3")
                p = int(input())
                pick = self.pick_sticks(p)
                print("Player picks %2d sticks" %p)
                self.turn = False
            else:
                # computer picks sticks
                c = strategy.pick_sticks(self.n)
                pick = self.pick_sticks(c)
                print("Computer picks %2d sticks" %c)
                self.turn = True
            
            if pick > self.n:
                pick = self.n-1
            self.n-= pick
            print("Sticks picked: %2d\n" %pick)
                
        if self.turn:         #n=1 & it's your turn
            cowsay.pig('You lose!')
        else:                 #n=1 & it's computer's turn
            cowsay.cow('You win!')
    
    def welcome(self):
        cowsay.cow('Welcome to Quantum Sticks')
        

In [529]:
q = QuantumSticks()

  _________________________
| Welcome to Quantum Sticks |
                         \
                          \
                            ^__^
                            (oo)\_______
                            (__)\       )\/\
                                ||----w |
                                ||     ||


In [530]:
q.begin_game()

You can put some sticks in superposition.
Please enter comma separated values: such as 1,12,5,21,..
12,13,15,19,20
-------------------------------------------
Sticks left: 25
Enter the number of sticks you want to pick- 1,2,3
3
Player picks  3 sticks
Sticks picked:  6

-------------------------------------------
Sticks left: 19
Computer picks  3 sticks
Sticks picked:  6

-------------------------------------------
Sticks left: 13
Enter the number of sticks you want to pick- 1,2,3
2
Player picks  2 sticks
Sticks picked:  2

-------------------------------------------
Sticks left: 11
Computer picks  3 sticks
Sticks picked:  2

-------------------------------------------
Sticks left:  9
Enter the number of sticks you want to pick- 1,2,3
3
Player picks  3 sticks
Sticks picked:  2

-------------------------------------------
Sticks left:  7
Computer picks  3 sticks
Sticks picked:  2

-------------------------------------------
Sticks left:  5
Enter the number of sticks you want to pick- 1,2

In [505]:
q = QuantumSticks()

  _________________________
| Welcome to Quantum Sticks |
                         \
                          \
                            ^__^
                            (oo)\_______
                            (__)\       )\/\
                                ||----w |
                                ||     ||
Circuit created


In [506]:
q.begin_game()

You can put some sticks in superposition.
Please enter comma separated values: such as 1,12,5,21,..
12,13,1,7,8,9


In [507]:
q.draw_circuit()

             ┌───┐ ░ ┌───┐ ░      
 q28_0: ─|0>─┤ X ├─░─┤ H ├─░───■──
             └───┘ ░ └───┘ ░ ┌─┴─┐
 q28_1: ─|0>───────░───────░─┤ X ├
             ┌───┐ ░       ░ └───┘
 q28_2: ─|0>─┤ X ├─░───────░───■──
             └───┘ ░       ░ ┌─┴─┐
 q28_3: ─|0>───────░───────░─┤ X ├
             ┌───┐ ░       ░ └───┘
 q28_4: ─|0>─┤ X ├─░───────░───■──
             └───┘ ░       ░ ┌─┴─┐
 q28_5: ─|0>───────░───────░─┤ X ├
             ┌───┐ ░ ┌───┐ ░ └───┘
 q28_6: ─|0>─┤ X ├─░─┤ H ├─░───■──
             └───┘ ░ ├───┤ ░ ┌─┴─┐
 q28_7: ─|0>───────░─┤ H ├─░─┤ X ├
             ┌───┐ ░ ├───┤ ░ └───┘
 q28_8: ─|0>─┤ X ├─░─┤ H ├─░───■──
             └───┘ ░ └───┘ ░ ┌─┴─┐
 q28_9: ─|0>───────░───────░─┤ X ├
             ┌───┐ ░       ░ └───┘
q28_10: ─|0>─┤ X ├─░───────░───■──
             └───┘ ░ ┌───┐ ░ ┌─┴─┐
q28_11: ─|0>───────░─┤ H ├─░─┤ X ├
             ┌───┐ ░ ├───┤ ░ └───┘
q28_12: ─|0>─┤ X ├─░─┤ H ├─░───■──
             └───┘ ░ └───┘ ░ ┌─┴─┐
q28_13: ─|0>───────░───────░─┤ X ├
             ┌───┐ ░

In [508]:
q.play_game()

-------------------------------------------
Sticks left: 25
Enter the number of sticks you want to pick- 1,2,3
2
Player picks  2 sticks
Sticks picked:  0

-------------------------------------------
Sticks left: 25
Computer picks  2 sticks
Sticks picked:  1

-------------------------------------------
Sticks left: 24
Enter the number of sticks you want to pick- 1,2,3
3
Player picks  3 sticks
Sticks picked:  5

-------------------------------------------
Sticks left: 19
Computer picks  2 sticks
Sticks picked:  2

-------------------------------------------
Sticks left: 17
Enter the number of sticks you want to pick- 1,2,3
1
Player picks  1 sticks
Sticks picked:  2

-------------------------------------------
Sticks left: 15
Computer picks  2 sticks
Sticks picked:  1

-------------------------------------------
Sticks left: 14
Enter the number of sticks you want to pick- 1,2,3
3
Player picks  3 sticks
Sticks picked:  0

-------------------------------------------
Sticks left: 14
Computer